# MSCF 46982 Market Microstructure and Algorithmic Trading

Fall 2025 Mini 2

Introduction to ICE, Spiderrock and Lobster data

Copyright &copy; 2025 Nick Psaris. All Rights Reserved

# TOC
- [Introduction](#Introduction)
- [Attribution](#Attribution)
- [Initialize](#Initialize)
- [Tables](#Tables)
- [TAQ](#TAQ)
- [CME](#CME)
- [Blockstream](#Blockstream)
- [Spiderrock](#Spiderrock)
- [Lobster](#Lobster)


# Introduction

This notebook introduces the Kdb+ database loaded with ICE and
Spiderrock data and includes:
- US Equity L1
- US CME Future L1
- Global Crypto Currencies L1
- US Option Time & Sales

All data covers the month of **December 2023** and therefore allows
**cross-market** and **cross-product** comparisons.


# Attribution
- Equity and Crypto market data provided by [ICE Data Services][]
- Crypto data feed sourced from [Blockstream][]
- Option market data provided by [Spiderrock][]
- **Any saved or downloaded data must be immediately removed from
  personal devices after completion of the course**


[ICE Data Services]: https://www.theice.com/market-data/cryptocurrencies "ICE Data Services"
[Blockstream]: https://blockstream.com/cryptofeed "Blockstream"
[Spiderrock]: https://spiderrock.net/data/historical-data-analytics/

# Initialize
- We start by initializing the number of rows and columns displayed as
  well as the database handle `h` and sample data date `dt`


In [1]:
import os
os.environ['PYKX_JUPYTERQ'] = 'true'
os.environ['PYKX_4_1_ENABLED'] = 'true'
import pykx as kx


PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


In [2]:
\c 15 120
/ windows and mac/linux use different environment variables
home:`HOME`USERPROFILE "w"=first string .z.o
upf:0N!` sv (hsym`$getenv home),`cmu_userpass.txt
h:`$":tcps://tpr-mscf-kx.tepper.cmu.edu:5002:",first read0 upf
dt:2023.12.01

`:/Users/nick/cmu_userpass.txt


# Tables
| Table     | Description                                                         |
|-----------|---------------------------------------------------------------------|
| ccy       | Mapping from three-letter Crypto currency code to the currency name |
| cryptomas | Security master table with Crypto-specific columns                  |
| daily     | Daily summary of TAQ, CME and Crypto feeds                          |
| ex        | Mapping from TAQ and Crypto exchanges (ex) to their name            |
| mas       | Security master table for TAQ, CME and Crypto feeds                 |
| nbbo      | Best bid and offer table for TAQ and CME feeds                      |
| opracode  | Mapping from Spiderrock prtType to OPRA code trade description      |
| opracols  | Column descriptions for the opratrade table                         |
| opradmm   | Exchange-specific Designated Market Maker (DMM) for each stock      |
| opratrade | Option trade table with extended analytics sourced from Spiderrock  |
| quote     | Bid and offer table for the TAQ, CME and Crypto feeds               |
| trade     | Trade table for TAQ, CME and Crypto feeds                           |




In [3]:
h"tables[]"

`ccy`cryptomas`daily`ex`handle`mas`nbbo`opracode`opracols`opradmm`opratrade`quote`trade


# TAQ

Trade and Quote data for [Tapes A, B, and C][]

[Tapes A, B, and C]: https://intrinio.com/blog/understanding-nyse-cta-tapes-a-b-and-c


## US Equity Exchanges
- US Equity exchanges are identified by a single lower-case letter

In [4]:
h ({select from ex where ex like x};"[a-z]")

ex name                                         
------------------------------------------------
a  "NYSE American, LLC (NYSE American)"         
b  "NASDAQ OMX BX, Inc.(NASDAQ OMX BX)"         
c  "NYSE National, Inc(NYSE National)"          
d  "FINRA Alternative Display Facility (ADF)"   
h  "MIAX Pearl, LLC (MIAX)"                     
j  "Cboe EDGA Exchange(Cboe EDGA)"              
k  "Cboe EDGX Exchange (Cboe EDGX)"             
l  "Long-Term Stock Exchange, Inc. (LTSE)"      
m  "Chicago Stock Exchange, Inc. (NYSE Chicago)"
n  "New York Stock Exchange, LLC(NYSE)"         
..


## US Equity Daily Summaries
- US Equities symbols do not have colons (CME Futures and
  Crypto-Currencies do)


In [5]:
h ({[dt;symre]10#select from daily where date = dt, not sym like symre};dt;"*:*")

date       sym    open    high    low     close price        size    n    
--------------------------------------------------------------------------
2023.12.01 A      127.67  129.45  126.43  129   2.216574e+08 1729631 31888
2023.12.01 AA     26.96   28.015  26.5101 27.71 1.509705e+08 5462935 49764
2023.12.01 AACT   10.3701 10.4    10.3701 10.4  7004.055     675     6    
2023.12.01 AAIC   4.69    4.78    4.66    4.78  844738.7     178116  1485 
2023.12.01 AAIC-B 20.9399 20.9399 20.47   20.47 8046.531     391     2    
2023.12.01 AAIC-C 24.26   24.4592 24.23   24.39 25729.89     1060    21   
2023.12.01 AAIN   23.58   23.68   23.56   23.68 6552.73      277     7    
2023.12.01 AAN    8.81    9.5     8.63    9.5   6624178      713217  9054 
2023.12.01 AAP    50.85   54.44   50.406  54.1  1.173245e+08 2211772 33709
2023.12.01 AAT    20.14   21.18   19.94   21.1  1.138829e+07 542714  5774 


## US Equities With Most Daily Transaction

In [6]:
h ({[dt;symre]10#update pct:n%sum n from desc 1!select sym,n from daily where date = dt, not sym like symre};dt;"*[:]*")

sym | n       pct        
----| -------------------
TSLA| 1293937 0.01649408 
SPY | 606486  0.007731003
NVDA| 537782  0.006855219
MSFT| 516942  0.006589567
AAPL| 515546  0.006571772
ALT | 491655  0.006267228
PFE | 478612  0.006100967
QQQ | 377502  0.004812096
GGE | 376944  0.004804984
IWM | 368547  0.004697945


## US Equity Trades
- Each trade lists the exchange it was transacted on

In [7]:
h ({[dt;syms]10#select from trade where date = dt, sym in syms};dt;`BAC)

date       sym time                          price size cond ex
---------------------------------------------------------------
2023.12.01 BAC 2023.12.01D09:00:00.012680000 30.6  6    9    p 
2023.12.01 BAC 2023.12.01D09:01:10.366470000 30.54 12   9    p 
2023.12.01 BAC 2023.12.01D09:02:54.957150000 30.54 79   9    p 
2023.12.01 BAC 2023.12.01D09:03:21.089430000 30.54 9    9    p 
2023.12.01 BAC 2023.12.01D09:03:28.334880000 30.56 1    9    t 
2023.12.01 BAC 2023.12.01D09:06:24.446930000 30.54 2    9    k 
2023.12.01 BAC 2023.12.01D09:09:47.051330000 30.54 26   9    k 
2023.12.01 BAC 2023.12.01D09:09:47.051500000 30.54 172  20   k 
2023.12.01 BAC 2023.12.01D09:09:49.060400000 30.56 15   9    k 
2023.12.01 BAC 2023.12.01D09:12:37.893320000 30.6  1    9    t 


## US Equity Exchange Quotes
- Each quote lists the exchange it was quoted on

In [8]:
h ({[dt;syms]10#select from quote where date = dt, sym in syms};dt;`BAC)

date       sym time                          bid ask   bsize asize cond ex
--------------------------------------------------------------------------
2023.12.01 BAC 2023.12.01D09:00:00.007120000     32.9        10    0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007140000     32.8        2     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007170000     32          2     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007170000     30.75       6     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007190000     30.75       6     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007440000     30.75       6     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007540000     30.75       7     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007620000     30.71       1     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007710000     30.71       1     0    k 
2023.12.01 BAC 2023.12.01D09:00:00.007800000 18  30.71 1     1     0    k 


## US Equity Consolidated Quotes
- Each consolidated quote lists the exchange the best bid and ask came from

In [9]:
h ({[dt;syms]10#select from nbbo where date = dt, sym in syms};dt;`BAC)

date       sym time                          bid   bsize bex ask   asize aex
----------------------------------------------------------------------------
2023.12.01 BAC 2023.12.01D09:00:00.007120000                 32.9  10    k  
2023.12.01 BAC 2023.12.01D09:00:00.007140000                 32.8  2     k  
2023.12.01 BAC 2023.12.01D09:00:00.007170000                 32    2     k  
2023.12.01 BAC 2023.12.01D09:00:00.007170000                 30.75 6     k  
2023.12.01 BAC 2023.12.01D09:00:00.007540000                 30.75 7     k  
2023.12.01 BAC 2023.12.01D09:00:00.007620000                 30.71 1     k  
2023.12.01 BAC 2023.12.01D09:00:00.007800000 18    1     k   30.71 1     k  
2023.12.01 BAC 2023.12.01D09:00:00.007810000 24.95 1     k   30.71 1     k  
2023.12.01 BAC 2023.12.01D09:00:00.007840000 27    4     k   30.71 1     k  
2023.12.01 BAC 2023.12.01D09:00:00.007870000 29    4     k   30.71 1     k  


# CME
- Futures trades and quotes for contracts traded on GLBX

## Root for Top CME Futures Contracts
| Root | Description                             |
|------|-----------------------------------------|
| MNQ  | Micro E-Mini Nasdaq-100 Index Futures   |
| MES  | Micro E-Mini S&P-500 Index Futures      |
| ES   | E-Mini S&P-500 Index Futures            |
| NQ   | E-Mini Nasdaq-100 Index Futures         |
| RTY  | E-Mini Russell-2000 Index Futures       |
| MTK  | Micro E-Mini Russell-2000 Index Futures |
| EMD  | E-Mini S&P-Midcap-400 Index Futures     |


## CME Futures Daily Summaries

In [10]:
h ({[dt;symre]10#select from daily where date = dt, sym like symre};dt;"F2:*")

date       sym        open    high    low     close   price        size  n    
------------------------------------------------------------------------------
2023.12.01 F2:E7\H24  1.0942  1.0962  1.0881  1.093   414.0072     379   202  
2023.12.01 F2:E7\Z23  1.0894  1.0919  1.0835  1.089   5614.058     5158  3248 
2023.12.01 F2:EGT\Z23 0.66    0.69    0.66    0.69    59.38        88    8    
2023.12.01 F2:EMD\Z23 2567.3  2630    2554.7  2628.3  4.698243e+07 18060 14348
2023.12.01 F2:EMT\Z23 3       3.15    3       3.15    643.8        212   6    
2023.12.01 F2:ESG\Z23 406.34  408.66  405.26  408.2   292645.3     719   380  
2023.12.01 F2:EST\Z23 7.5     8       6.9     7.25    366299.7     47300 2013 
2023.12.01 F2:ES\H24  4622.25 4657.75 4612.75 4654.75 1.191164e+08 25674 8413 
2023.12.01 F2:ES\M24  4674.25 4705.75 4665    4705.75 900110.2     192   98   
2023.12.01 F2:ES\U24  4720.25 4720.25 4720.25 4720.25 160488.5     34    5    


## CME Futures Contracts With Most Daily Transactions


In [11]:
h ({[dt;symre]10#update pct:n%sum n from desc 1!select sym, n from daily where date = dt, sym like symre};dt;"F2:*")

sym       | n      pct        
----------| ------------------
F2:MNQ\Z23| 559884 0.2914312  
F2:ES\Z23 | 471273 0.2453074  
F2:NQ\Z23 | 375633 0.1955248  
F2:MES\Z23| 271576 0.1413609  
F2:RTY\Z23| 132822 0.06913661 
F2:M2K\Z23| 49272  0.0256471  
F2:EMD\Z23| 14348  0.007468432
F2:MNQ\H24| 13714  0.007138422
F2:ES\H24 | 8413   0.004379141
F2:MES\H24| 8400   0.004372374

## CME December Futures Contracts Open Interest

In [12]:
h ({[dt;e]10#`oi xdesc select from mas where date = dt, expir = e};dt;2023.12.15)

date       sym        ccy isin cusip itype lotsize contractsize ticksize oi      expir      exchsym mic 
--------------------------------------------------------------------------------------------------------
2023.12.01 F2:ES\Z23  USD ""   ""    1280          50           "0.25"   2240364 2023.12.15 ESZ3    GLBX
2023.12.01 F2:RTY\Z23 USD ""   ""    1280          50           "0.1"    548175  2023.12.15 RTYZ3   GLBX
2023.12.01 F2:NQ\Z23  USD ""   ""    1280          20           "0.25"   279273  2023.12.15 NQZ3    GLBX
2023.12.01 F2:MES\Z23 USD ""   ""    1280          5            "0.25"   150080  2023.12.15 MESZ3   GLBX
2023.12.01 F2:MNQ\Z23 USD ""   ""    1280          2            "0.25"   99587   2023.12.15 MNQZ3   GLBX
2023.12.01 F2:EMD\Z23 USD ""   ""    1280          100          "0.1"    39868   2023.12.15 EMDZ3   GLBX
2023.12.01 F2:XAE\Z23 USD ""   ""    1280          100          "0.1"    37404   2023.12.15 XAEZ3   GLBX
2023.12.01 F2:XAU\Z23 USD ""   ""    1280          100 

## CME Futures Trades
- No exchanges are listed because there is only one (GLBX)
- The backslash on the symbol needs to be escaped within a string

In [13]:
h ({[dt;syms]10#select from trade where date = dt, sym in syms};dt;`$"F2:ES\\Z23")

date       sym       time                          price   size cond ex
-----------------------------------------------------------------------
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.013320000 4571.75 18   0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.014090000 4572    1    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.014310000 4572    3    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.016330000 4572    1    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.020310000 4572    1    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.021620000 4572    1    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.029050000 4572    1    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.030810000 4571.75 9    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.035500000 4571.75 1    0      
2023.12.01 F2:ES\Z23 2023.11.30D23:00:00.037900000 4571.75 1    0      

## CME Futures Exchange Quotes
- No exchanges are listed because there is only one (Globex)

In [14]:
h ({[dt;syms]10#select from quote where date = dt, sym in syms};dt;`$"F2:ES\\Z23")

date       sym       time                          bid     ask     bsize asize cond ex
--------------------------------------------------------------------------------------
2023.12.01 F2:ES\Z23 2023.11.30D22:45:01.019800000         4572          18           
2023.12.01 F2:ES\Z23 2023.11.30D22:45:52.132360000         4572          15           
2023.12.01 F2:ES\Z23 2023.11.30D22:45:53.029320000         4572          14           
2023.12.01 F2:ES\Z23 2023.11.30D22:46:32.884440000 4572    4572    2     14           
2023.12.01 F2:ES\Z23 2023.11.30D22:47:42.909240000 4572    4572    2     15           
2023.12.01 F2:ES\Z23 2023.11.30D22:51:19.052990000 4571.5  4572    17    15           
2023.12.01 F2:ES\Z23 2023.11.30D22:51:19.052990000 4571.75 4572    2     15           
2023.12.01 F2:ES\Z23 2023.11.30D22:51:19.052990000 4571.75 4571.75 2     2            
2023.12.01 F2:ES\Z23 2023.11.30D22:51:32.025880000 4571.5  4571.75 17    2            
2023.12.01 F2:ES\Z23 2023.11.30D22:51:32.02

## CME Futures Consolidated Quotes
- There is no consolidated feed because there is only one exchange

In [15]:
h ({[dt;syms]select from nbbo where date = dt, sym in syms};dt;`$"F2:ES\\Z23")

date sym time bid bsize bex ask asize aex
-----------------------------------------


# Blockstream

## Crypto Currencies

In [16]:
h"ccy"

ccy currency    
----------------
ADA "Cardano"   
ALG "Algorand"  
AMP "Ampleforth"
ANK "Ankr"      
ANT "Aragon"    
APE "ApeCoin"   
ARB "Arbitrum"  
ARD "Ardor"     
ARS "Argentine" 
ATO "Cosmos"    
..


## Crypto Currency Exchanges

In [17]:
h ({[exre]select from ex where ex like exre};"???")

ex  name                    
----------------------------
BIN "Binance.com"           
BIU "Binance_US"            
BBK "Bitbank"               
BFX "Bitfinex (Derivatives)"
BFL "Bitflyer (Derivatives)"
BMX "Bitmex (Derivatives)"  
BSO "Bitso"                 
BST "BitStamp"              
BTC "BTCBox"                
BTK "BTCTurk"               
..


## Crypto-Specific Security Master
- Spot currencies have symbols starting with "X:S" followed by 6
  characters representing the two 3-letter currencies
- Exchange-specific symbols are followed by an "@" and the
  three-letter crypto exchange
- Provides the closing price across multiple regions
- Example query demonstrates records for US dollar (USD) and TerraUSD
  (UST) denominated securities


In [18]:
h ({[dt;symre]10#select from cryptomas where date = dt, sym like symre};dt;"X:S???US[DT]")

date       sym       contractsize closejp    closejpdt  closesg    closesgdt  closegb    closegbdt  closeus    closeu..
---------------------------------------------------------------------------------------------------------------------..
2023.12.01 X:SADAUSD              0.3789656  2023.12.01 0.3790709  2023.12.01 0.3815058  2023.12.01 0.3844787  2023.1..
2023.12.01 X:SADAUST              0.3780379  2023.12.01 0.37817    2023.12.01 0.3808084  2023.12.01 0.3843865  2023.1..
2023.12.01 X:SALGUSD              0.1346279  2023.12.01 0.1336805  2023.12.01 0.1367192  2023.12.01 0.1387333  2023.1..
2023.12.01 X:SALGUST              0.1343705  2023.12.01 0.133697   2023.12.01 0.136606   2023.12.01 0.1388902  2023.1..
2023.12.01 X:SAMPUST              1.175805   2023.12.01 1.179243   2023.12.01 1.197646   2023.12.01 1.199497   2023.1..
2023.12.01 X:SANKUSD              0.02555762 2023.12.01 0.02557502 2023.12.01 0.02545056 2023.12.01 0.02576227 2023.1..
2023.12.01 X:SANKUST              0.0256

## Crypto Currency Daily Summaries


In [19]:
h ({[dt;symre]10#select from daily where date = dt, sym like symre};dt;"X:S??????")

date       sym       open      high      low       close     price        size         n     
---------------------------------------------------------------------------------------------
2023.12.01 X:SADAETH 0.0001837 0.0001846 0.0001801 0.0001835 675.3722     3702210      3057  
2023.12.01 X:SADAEUR 0.34522   0.3548    0.34311   0.3518    544845.5     1555482      3713  
2023.12.01 X:SADAGBP 0.3016    0.3091    0.30143   0.3089    9996.265     32738.76     89    
2023.12.01 X:SADAJPY 55.719    56.707    55.36     56.212    3.346764e+07 595233       839   
2023.12.01 X:SADATRY 10.87     11.2      10.82     11.097    2.022525e+07 1832821      3881  
2023.12.01 X:SADAUDC 0.37536   0.3858    0.37318   0.38199   1392371      3660603      3890  
2023.12.01 X:SADAUSD 0.37579   0.4023993 0.37292   0.38198   6549598      1.721391e+07 13515 
2023.12.01 X:SADAUST 0.374744  0.38567   0.37315   0.3825    4.146894e+07 1.08966e+08  244063
2023.12.01 X:SADAXBT 9.96e-06  1.01e-05  9.8e-06   9.88e-06 

## Crypto Currencies With Most Daily Transactions

In [20]:
h ({[dt;symre]10#update pct:n%sum n from desc 1!select sym,n from daily where date = dt, sym like symre};dt;"X:S??????")

sym      | n       pct       
---------| ------------------
X:SXBTUST| 3696592 0.1865164 
X:SETHUST| 998019  0.05035636
X:SSOLUST| 492312  0.02484025
X:SLUNUST| 435822  0.02198997
X:SAVAUST| 398390  0.02010129
X:SXDGUST| 393716  0.01986546
X:SXRPUST| 383025  0.01932603
X:SMATUST| 318944  0.01609274
X:SAVEUST| 286616  0.01446159
X:SUDCUST| 270753  0.0136612 

## Crypto Currency Trades
- Each trade lists the exchange it was transacted on


In [21]:
h ({[dt;symre]10#select from trade where date = dt, sym = symre};dt;`$"X:SXBTUST")

date       sym       time                          price    size    cond ex 
----------------------------------------------------------------------------
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37760    0.00168 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37760    0.00242 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37760    0.00243 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37760    0.00225 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37771.07 0.00016 0    BIU
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.150600000 37760.01 0.00104 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.150600000 37760.01 0.00191 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.150600000 37760.01 0.00074 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.150600000 37760.01 0.00106 0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.150600000 37760.01 0.00237 0    BIN

## Crypto Currency Exchange Quotes
- Each quote lists the exchange it was transacted quoted on

In [22]:
h ({[dt;syms]10#select from quote where date = dt, sym in syms};dt;`$"X:SXBTUST")

date       sym       time                          bid      ask      bsize      asize     cond ex 
--------------------------------------------------------------------------------------------------
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.070300000 37283.8  38039.8  0.7591857  0.7591857 0    CEX
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.070300000 37755.85 37755.86 0.60443    2.617725  0    HBI
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.099800000 37755.85 37755.86 0.606697   2.617725  0    HBI
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.099800000 37755.85 37755.86 0.60443    2.617725  0    HBI
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37745    37801    0.03333333 0.1509453 0    BSO
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.138700000 37757.6  37771.07 0.008      0.01445   0    BIU
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.150600000 37760    37760.01 4.68563    10.58959  0    BIN
2023.12.01 X:SXBTUST 2023.11.30D22:55:00.260700000 37757.6  37771.07 0.008      0.01461   0    BIU
2023.12.01

## Crypto Currency Consolidated Quotes
- There is **no** consolidated feed that lists the best bid/ask across exchanges

In [23]:
h ({[dt;syms]select from nbbo where date = dt, sym in syms};dt;`$"X:SXBTUST")

date sym time bid bsize bex ask asize aex
-----------------------------------------


# Spiderrock
- US Option trades sourced from [OPRA][] with additional analytics
  provided by [Spiderrock][]

[OPRA]: https://www.opraplan.com/
[Spiderrock]: https://spiderrock.net/data/historical-data-analytics/

## US Option Tickers With Most Daily Transactions

In [24]:
h ({[dt]10#update pct:n%sum n from desc select n:count i by ticker_tk from opratrade where date = dt};dt)

ticker_tk| n      pct       
---------| -----------------
SPY      | 760304 0.1277073 
SPX      | 606882 0.1019372 
TSLA     | 471818 0.07925068
QQQ      | 293237 0.04925466
NVDA     | 254290 0.04271278
IWM      | 127061 0.02134228
MSFT     | 106286 0.01785273
AAPL     | 85728  0.01439963
META     | 77479  0.01301405
AMD      | 70390  0.01182332


## US Option Exchange-Specific DMM for each Ticker
The `ticker`, `exchange`, and `dmm` are available in the
[opradmm.csv][] file and `opradmm` table.

[opradmm.csv]: https://tpr-mscf-kx.tepper.cmu.edu:5002/opradmm.csv


In [25]:
h ({.util.pivot 2!opradmm};`)

ticker| AMEX          AMEX E-Spec 1 AMEX E-Spec 2 CBOE      EDGO    EMLD           GMNI           ISE            MCRY..
------| -------------------------------------------------------------------------------------------------------------..
A     | XR Securities SIG                         SIG       Citadel WEX                           Citadel        SIG ..
AA    | XR Securities Citadel                     Citadel   Citadel SIG                           SIG            Cita..
AACQ  | Group One     SIG                         Belvedere SIG     Citadel        SIG            Citadel        Belv..
AAIC  | Citadel       SIG                         WEX       Citadel Morgan Stanley SIG            WEX            Morg..
AAL   | SIG           Citadel                     SIG       IMC     IMC            Morgan Stanley Citadel        Cita..
AAN   | Belvedere     Citadel                                                                                        ..
AAOI  | XR Securities SIG               

## Percent of Total Seats Held by Each DMM


In [26]:
h ({10#update pct:n%sum n from desc select n:count i by dmm from opradmm};`)

dmm           | n     pct       
--------------| ----------------
Citadel       | 12937 0.32523   
SIG           | 11227 0.2822414 
WEX           | 6153  0.1546835 
Morgan Stanley| 2986  0.07506662
Belvedere     | 1421  0.03572326
Two Sigma     | 1379  0.0346674 
Group One     | 1254  0.03152496
GTS           | 899   0.02260043
IMC           | 638   0.01603902
XR Securities | 442   0.01111167


## US Option Trade Type

The `prtType` column provides the OPRA code which defines the type of
last sale (a.k.a. trade) message. The values are delivered as integers
but should be converted to characters to map to the values specified
in the [OPRA Pillar Output Specification][].

The `code`, `mnemonic`, short `descr` and long `description` are
available in the [opracode.tsv][] file and `opracode` table.


[OPRA Pillar Output Specification]: https://cdn.opraplan.com/documents/OPRA_Pillar_Output_Specification.pdf

[opracode.tsv]: https://tpr-mscf-kx.tepper.cmu.edu:5002/opracode.tsv

In [27]:
h"opracode"

code mnemonic desc description                                                                                       ..
---------------------------------------------------------------------------------------------------------------------..
A    CANC     ""   "Transaction previously reported (other than as the last or opening report for the particular opti..
B    OSEQ     ""   "Transaction is being reported late and is out of sequence; i.e., later transactions have been rep..
C    CNCL     ""   "Transaction is the last reported for the particular option contract and is now cancelled."       ..
D    LATE     ""   "Transaction is being reported late, but is in the correct sequence; i.e., no later transactions h..
E    CNCO     ""   "Transaction was the first one (opening) reported this day for the particular option contract. Alt..
F    OPEN     ""   "Transaction is a late report of the opening trade and is out of sequence; i.e., other transaction..
G    CNOL     ""   "Transaction was the 

## US Option Trade Types With Most Daily Transactions

We can list the most frequent trade types, convert the `prtType` to a
character and join with the `opracode` table to describe each one.


In [28]:
codefreq:{[dt]
 t:desc select n:count i by code:"c"$prtType from opratrade where date = dt;
 t:update pct:n%sum n from t;
 t:t lj 1!opracode;
 t}
h (codefreq;dt)


code| n       pct          mnemonic desc                                                    description              ..
----| ---------------------------------------------------------------------------------------------------------------..
I   | 3328600 0.5591008    AUTO     ""                                                      "Transaction was executed..
a   | 921021  0.1547028    SLAN     "Single Leg Auction"                                    "Non ISO Transaction was ..
f   | 782963  0.1315133    MLET     "Multi Leg auto-electronic trade"                       "Transaction represents a..
g   | 380980  0.06399274   MLAT     "Multi Leg Auction"                                     "Transaction was the exec..
S   | 349298  0.05867115   ISOI     ""                                                      "Transaction was the exec..
j   | 156938  0.02636068   MESL     "Multi Leg auto-electronic trade against single leg(s)" "Transaction represents a..
n   | 11355   0.001907285  TLET     "Sto

## US Option Trades

In [29]:
h ({[dt;syms]10#select from opratrade where date = dt, ticker_tk in syms};dt;`BAC)

date       ticker_tk okey_ts okey_yr okey_mn okey_dy okey_xx okey_cp timestamp                     prtNumber tradingS..
---------------------------------------------------------------------------------------------------------------------..
2023.12.01 BAC       NMS     2023    12      1       31.5    Call    2023.12.01D14:30:02.966109000 13587     RegularM..
2023.12.01 BAC       NMS     2024    1       19      42      Call    2023.12.01D14:30:03.164319000 14161     RegularM..
2023.12.01 BAC       NMS     2024    1       19      42      Call    2023.12.01D14:30:03.164319000 14164     RegularM..
2023.12.01 BAC       NMS     2024    1       19      42      Call    2023.12.01D14:30:03.164319000 14165     RegularM..
2023.12.01 BAC       NMS     2023    12      15      31      Call    2023.12.01D14:30:03.436575000 14896     RegularM..
2023.12.01 BAC       NMS     2024    1       19      30      Call    2023.12.01D14:30:03.469606000 14997     RegularM..
2023.12.01 BAC       NMS     2024    6  

## US Option Trade Column Description

The `column` and `description` are available in the [opracols.tsv][]
file and `opracols` table.

[opracols.tsv]: https://tpr-mscf-kx.tepper.cmu.edu:5002/opracols.tsv

Each trade includes the:
- Trade details
- Option details (strike/expiration/etc)
- Option exchange and NBBO bid/ask at time of trade
- Underlying bid/ask at time of trade
- Security greeks (delta/gamma/theta/etc) at time of trade
- Volatility surface parameters at time of trade
- Option inputs, prices, and P&L at 1 and 10 minutes after the trade


In [30]:
h "opracols"

column    description                                                                      
-------------------------------------------------------------------------------------------
okey_at   "Option underlying asset type"                                                   
okey_ts   "Option ticker source"                                                           
okey_tk   "Option underlying symbol"                                                       
okey_yr   "Option expiration year"                                                         
okey_mn   "Option expiration month"                                                        
okey_dy   "Option expiration day"                                                          
okey_xx   "Option strike"                                                                  
okey_cp   "Option call/put indicator"                                                      
timestamp "Timestamp of last update to record - UTC"                            

# Lobster

In [31]:
\c 15 120
h:`$":tcps://tpr-mscf-kx.tepper.cmu.edu:5005:",first read0 ` sv (hsym`$getenv`HOME),`cmu_userpass.txt
dt:2012.06.21

In [32]:
h "tables[]"

`s#`book1`book10`book30`book5`book50


In [33]:
book:{[dt;tbl;syms]
 t:select from tbl where date=dt,sym in syms;
 t}
h (book;2012.06.21;`book5;`AAPL)

date       sym  time                 mtype oid      size price  side ask                                asize        ..
---------------------------------------------------------------------------------------------------------------------..
2012.06.21 AAPL 0D09:30:00.004241176 1     16113575 18   585.33 1    585.94 585.98 586.1  586.89 586.95 200 200 200 3..
2012.06.21 AAPL 0D09:30:00.004260640 1     16113584 18   585.32 1    585.94 585.98 586.1  586.89 586.95 200 200 200 3..
2012.06.21 AAPL 0D09:30:00.004447484 1     16113594 18   585.31 1    585.94 585.98 586.1  586.89 586.95 200 200 200 3..
2012.06.21 AAPL 0D09:30:00.025551909 1     16120456 18   585.91 -1   585.91 585.94 585.98 586.1  586.89 18  200 200 2..
2012.06.21 AAPL 0D09:30:00.025579546 1     16120480 18   585.92 -1   585.91 585.92 585.94 585.98 586.1  18  18  200 2..
2012.06.21 AAPL 0D09:30:00.025613151 1     16120503 18   585.93 -1   585.91 585.92 585.93 585.94 585.98 18  18  18  2..
2012.06.21 AAPL 0D09:30:00.201517942 1  